In [17]:
%load_ext autoreload 
%autoreload 2
import sys
sys.tracebacklimit = 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from pettingzoo.test import parallel_api_test
from custom_gym import CustomGymEnviornment

In [19]:
from core.agent import *
from core.world import * 
from core.map import *
from core.render import * 
from core.env_params import * 
from core.skill import * 
from core.models import *
from core.env_params import *

In [20]:
def initialize_swarm(world : World):
    swarm = [Agent() for i in range(SWARM_SIZE)]
    skill_initializer = DefaultSkillInitializer(num_skills = PRODUCT_TYPES)
    utility_initializer = UtilitySampler()

    for agent in swarm:
        skill_initializer.forward(agent)
        utility_initializer.forward(agent)
        world.add_agent(agent)
    swarm = initialize_positions_randomly(world, swarm)
    

In [21]:
world = World(dims = WORLD_DIMS,
              swarm_initialzier= initialize_swarm,
              resource_generator= RandomMapGenerator(RESOURCE_TYPES),
              energy_model=EnergyModel(),
              chemistry_model= ChemistryModel()
              )
world.reset()

In [22]:
custom_gym : CustomGymEnviornment = CustomGymEnviornment(world)

In [23]:
custom_gym.reset(42)

({<core.agent.Agent at 0x1dc3a012850>: {'vision': array([[0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0]])},
  <core.agent.Agent at 0x1dc3b3b2ad0>: {'vision': array([[0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 5, 5, 5, 0, 0],
          [0, 0, 5, 5, 5, 0, 0],
          [0, 0, 5, 5, 5, 0, 0]])},
  <core.agent.Agent at 0x1dc3b3b2650>: {'vision': array([[0, 0, 1, 1, 1, 1, 1],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0]])},
  <core.agent.Agent at 0x1dc3b3b2d50>: {'vision': array([[0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [

### Testing

In [24]:
parallel_api_test(custom_gym, num_cycles=1_000_000)

Passed Parallel API test
